# Assignment 2: IR

## Preparations
* Put all your imports, and path constants in the next cells
* Make sure all your path constants are **relative to** ***DATA_DIR*** and **NOT hard-coded** in your code.

In [1]:
!pip install whoosh
!pip install pytrec_eval
!pip install wget

  Running setup.py clean for pytrec-eval
Failed to build pytrec-eval
    Running setup.py install for pytrec-eval: started
    Running setup.py install for pytrec-eval: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\Users\harshp\anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\harshp\\AppData\\Local\\Temp\\pip-install-6e65zijo\\pytrec-eval\\setup.py'"'"'; __file__='"'"'C:\\Users\\harshp\\AppData\\Local\\Temp\\pip-install-6e65zijo\\pytrec-eval\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\harshp\AppData\Local\Temp\pip-wheel-yyod_otk'
       cwd: C:\Users\harshp\AppData\Local\Temp\pip-install-6e65zijo\pytrec-eval\
  Complete output (11 lines):
  Fetching trec_eval from https://github.com/usnistgov/trec_eval/archive/v9.0.8.tar.gz.
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\pytrec_eval
  copying py\__init__.py -> build\lib.win-amd64-3.8\pytrec

  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9686 sha256=238d2064a5e63bc90dbcb9933fa3934b9c5a979ea4da9bbb7eb685361f9c1b44
  Stored in directory: c:\users\harshp\appdata\local\pip\cache\wheels\bd\a8\c3\3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget


In [2]:
import wget
wget.download("https://github.com/MIE451-1513-2019/course-datasets/raw/master/government.zip", "government.zip")

'government.zip'

In [3]:
!unzip government.zip


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [4]:
# imports
# Put all your imports here

from whoosh import index, writing
from whoosh.fields import Schema, TEXT, KEYWORD, ID, STORED
from whoosh.analysis import *
from whoosh import qparser, query
from whoosh.qparser import QueryParser
from whoosh.analysis import CharsetFilter
from whoosh.support.charset import accent_map
import os.path

from pathlib import Path
import tempfile
import subprocess
import pytrec_eval
import wget
import nltk
from nltk.stem import *
from nltk.corpus import wordnet as wn
from whoosh import scoring
from whoosh.scoring import MultiWeighting, Frequency, TF_IDF
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV

ModuleNotFoundError: No module named 'pytrec_eval'

In [ ]:
DATA_DIR = "government"
#
# Put other path constants here
#

DOCUMENTS_DIR = os.path.join(DATA_DIR, "documents")

TOPIC_FILE = os.path.join(DATA_DIR, "gov.topics")
QRELS_FILE = os.path.join(DATA_DIR, "gov.qrels")

In [ ]:
DOCUMENTS_DIR

'government/documents'

## Question 1
Provide your text answers in the following two markdown cells

### Q1 (a): Provide answer to Q1 (a) here [markdown cell]
- map measure 

### Q1 (b): Provide answer to Q1 (b) here [markdown cell]
- map means mean average precision. If we get relevant results in the first few instances then we can say that the IR system is working good compared to other IR systems which might have the exact same precision but relevant documents are not at the initial results. So we are getting these differences through map measure. 

## Question 2

### Q2 (a): Write your code below

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q2, your query parser in QP_Q2, and your searcher in SEARCHER_Q2
def createIndex(schema):
    # Generate a temporary directory for the index
    indexDir = tempfile.mkdtemp()

    # create and return the index
    return index.create_in(indexDir, schema)

In [ ]:
# first, define a Schema for the index
mySchema = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = RegexTokenizer()))

# now, create the index at the path INDEX_DIR based on the new schema
INDEX_Q2 = createIndex(mySchema)

In [ ]:
def addFilesToIndex(indexObj, fileList):
    # open writer
    writer = writing.BufferedWriter(indexObj, period=None, limit=1000)

    try:
        # write each file to index
        for docNum, filePath in enumerate(fileList):
            with open(filePath, "r", encoding="utf-8") as f:
                fileContent = f.read()
                writer.add_document(file_path = filePath,
                                    file_content = fileContent,
                                    )

                # print status every 1000 documents
                if (docNum+1) % 1000 == 0:
                    print("already indexed:", docNum+1)
        print("done indexing.")

    finally:
        # close the index
        writer.close()

In [ ]:
# Build a list of files to index
filesToIndex = [str(filePath) for filePath in Path(DOCUMENTS_DIR).glob("**/*") if filePath.is_file()]

In [ ]:
filesToIndex[:10]

['government/documents/62/G00-62-0870952',
 'government/documents/62/G00-62-0415887',
 'government/documents/62/G00-62-2860802',
 'government/documents/62/G00-62-2252378',
 'government/documents/62/G00-62-2501423',
 'government/documents/62/G00-62-2530435',
 'government/documents/62/G00-62-3414482',
 'government/documents/62/G00-62-2770020',
 'government/documents/62/G00-62-2738054',
 'government/documents/62/G00-62-3597770']

In [ ]:
# count files to index
print("number of files:", len(filesToIndex))

number of files: 4078


In [ ]:
addFilesToIndex(INDEX_Q2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:

#INDEX_Q2 = createIndex(mySchema) # Replace None with your index for Q2
QP_Q2 = QueryParser("file_content", schema=INDEX_Q2.schema) # Replace None with your query parser for Q2
SEARCHER_Q2 = INDEX_Q2.searcher() # Replace None with your searcher for Q2

In [ ]:
# Is it empty?
print("Index is empty?", INDEX_Q2.is_empty())

# How many files indexed?
print("Number of indexed files:", INDEX_Q2.doc_count())

Index is empty? False
Number of indexed files: 4078


In [ ]:
# print the topic file
with open(TOPIC_FILE, "r") as f:
    print(f.read())

1 mining gold silver coal
2 juvenile delinquency
4 wireless communications
6 physical therapists
7 cotton industry
9 genealogy searches
10 Physical Fitness
14 Agricultural biotechnology
16 Emergency and disaster preparedness assistance
18 Shipwrecks
19 Cybercrime, internet fraud, and cyber fraud
22 Veteran's Benefits
24 Air Bag Safety
26 Nuclear power plants
28 Early Childhood Education



In [ ]:
sampleQuery = QP_Q2.parse("Emergency and disaster preparedness assistance")
sampleQueryResults = SEARCHER_Q2.search(sampleQuery, limit=None)

In [ ]:
sampleQueryResults

<Top 7 Results for And([Term('file_content', 'Emergency'), Term('file_content', 'and'), Term('file_content', 'disaster'), Term('file_content', 'preparedness'), Term('file_content', 'assistance')]) runtime=0.021278038000005495>

In [ ]:
# inspect the result:
# for each document print the rank and the score
for (docnum, result) in enumerate(sampleQueryResults):
    score = sampleQueryResults.score(docnum)
    fileName = os.path.basename(result["file_path"])
    print(fileName, docnum, score)

G00-34-3591274 0 34.092075576251
G00-05-0719078 1 32.195485675325315
G00-92-2053892 2 27.13176387951665
G00-70-2681284 3 26.574622331662233
G00-33-2857182 4 21.813916494265353
G00-51-3264753 5 10.948532555349608
G00-32-1907807 6 10.008862397603384


In [ ]:
def pyTrecEval(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
            print_line(measure, query_id, value)
    for measure in query_measures.keys():
        if measure == "runid":
              continue
        print_line(
            measure,
            'all',
            pytrec_eval.compute_aggregated_measure(
                measure,
                [query_measures[measure]
                 for query_measures in results.values()]))

In [ ]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2)


num_q                    1       1.0000
num_ret                  1       1.0000
num_rel                  1       5.0000
num_rel_ret              1       0.0000
map                      1       0.0000
gm_map                   1       -11.5129
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0000
iprec_at_recall_0.00     1       0.0000
iprec_at_recall_0.10     1       0.0000
iprec_at_recall_0.20     1       0.0000
iprec_at_recall_0.30     1       0.0000
iprec_at_recall_0.40     1       0.0000
iprec_at_recall_0.50     1       0.0000
iprec_at_recall_0.60     1       0.0000
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0000
P_30                     1       0.000

### Q2 (b): Provide answer to Q2 (b) here [markdown cell]
- map is 0.1971. The performance is not good. 

### Q2 (c): Provide answer to Q2(c) here [markdown cell]
- with topic IDs 18 and 24 whoosh system did very well as map is 1.
- with topic IDs 1,2,6,7,9,16, and 28 it did very badly as the map for these IDs is 0 whereas, for topic ID 19, it did not give any map value.  

## Question 3

In [ ]:
def printRelName(topicFile, qrelsFile, queryParser, searcher, id):
  with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()
  for topic in topics:
        topic_id, topic_phrase = tuple(topic.split(" ", 1))
        if topic_id == id:
          print("---------------------------Topic_id and Topic_phrase----------------------------------")
          print(topic_id, topic_phrase)
          topicQuery = queryParser.parse(topic_phrase)
          topicResults = searcher.search(topicQuery, limit=None)
          print("---------------------------Return documents----------------------------------")
          for (docnum, result) in enumerate(topicResults):
              score = topicResults.score(docnum)
              print("%s Q0 %s %d %lf test" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
          print("---------------------------Relevant documents----------------------------------")
          with open(qrelsFile, 'r') as f_qrel:
            qrels = f_qrel.readlines()
            for i in qrels:
              qid, _, doc, rel = i.rstrip().split(" ")
              if qid == id and rel == "1":
                print(i.rstrip())

In [ ]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q2, SEARCHER_Q2, "16")

---------------------------Topic_id and Topic_phrase----------------------------------
16 Emergency and disaster preparedness assistance
---------------------------Return documents----------------------------------
16 Q0 G00-34-3591274 0 34.092076 test
16 Q0 G00-05-0719078 1 32.195486 test
16 Q0 G00-92-2053892 2 27.131764 test
16 Q0 G00-70-2681284 3 26.574622 test
16 Q0 G00-33-2857182 4 21.813916 test
16 Q0 G00-51-3264753 5 10.948533 test
16 Q0 G00-32-1907807 6 10.008862 test
---------------------------Relevant documents----------------------------------
16 0 G00-03-0589290 1
16 0 G00-21-0494028 1
16 0 G00-21-2114990 1
16 0 G00-32-0551737 1
16 0 G00-86-3719816 1
16 0 G00-92-2974327 1
16 0 G00-99-0140748 1


In [ ]:
# define a reader object on the index
Reader_Q2 = INDEX_Q2.reader()

In [ ]:
#how many terms do we have?
print(Reader_Q2.field_length("file_content"))

2165181


In [ ]:
# how many documents have the phares "Emergency", "and", "disaster", "preparedness" and "assistance"
# in the field "file_content"?
print("# docs with 'Emergency'", Reader_Q2.doc_frequency("file_content", "Emergency"))
print("# docs with 'and'", Reader_Q2.doc_frequency("file_content", "and"))
print("# docs with 'disaster'", Reader_Q2.doc_frequency("file_content", "disaster"))
print("# docs with 'preparedness'", Reader_Q2.doc_frequency("file_content", "preparedness"))
print("# docs with 'assistance'", Reader_Q2.doc_frequency("file_content", "assistance"))

# docs with 'Emergency' 204
# docs with 'and' 3348
# docs with 'disaster' 62
# docs with 'preparedness' 26
# docs with 'assistance' 278


In [ ]:
# open false-positive file: 
with open("government/documents/70/G00-70-2681284", "r") as file:
  text = file.read()

print(text)

http://www.access.wa.gov/emergency/index.asp

   Access Washington

   Featured Sites    Public Services    Business    Education
   Government    Online Services    Employment
   Search    Site Information    Graphic Version    Email Lists    Home
     __________________________________________________________________

   Drought | Earthquake | Energy | Flood | Terrorism | Tsunami | Volcano |
   Wildfire | More state government news


   red and black graphic icon saying ALERT Emergency resources
   | Emergency resources | Child, adolescent mental health | Adult mental
   health | Special needs | Prepare for an emergency |
     __________________________________________________________________

   State services, resources
   State Operator
   1-800-321-2808, 8:00a.m. to 5:00p.m. (PST)

   Public and medical assistance, food stamps
   Community service offices
   Community Services Division Constituent Relations, 800-865-7801

   Traffic and travel
   Traffic and travel conditions
  

In [ ]:
# open false-negative file: 
with open("government/documents/92/G00-92-2974327", "r") as file:
  text = file.read()

print(text)

http://www.fema.gov/dizprepare.htm

     Project Impact Banner, this is an image map, please refer to text links
                                     below.

       [ home | feedback | library | privacy policy | search | site help |
                                  site index ]


     Disaster Information for Preparation
     __________________________________________________________________


       * Preparedness
       * Know Your Risks
       * Pets and Disasters
       * Flood Insurance



     This is the FEMA footer graphic at the bottom of the page

       FEMA 500 C Street, SW Washington, D.C. 20472 Phone: (202) 646-4600



### Q3 (a): Provide answer to Q3 (a) here [markdown cell]
- To answer this question, I am going to select query no.16 (Topic ID:16) - Emergency and disaster preparedness assistance. 
- For the selected query, the whoosh baseline system was not working up to the mark. The results of the printRelName function show that by passing this query to search relevant documents, it suggests wrong documents(False Positives).
- If we compare the results with ground truth results or relevant documents which the system should have suggested, we observe that it was not giving any of the relevant documents (False Negative).
### (1) : FALSE POSITIVES: 
- Below documents were highly ranked and given as the result of the chosen query. These all documents are false positives. 
- G00-34-3591274
- G00-05-0719078
- G00-92-2053892
- G00-70-2681284
- G00-33-2857182
- G00-51-3264753
- G00-32-1907807

 ### (2) : FALSE NEGATIVES: 
- Below documents should have highly ranked and given as the result for the chosen query. These documents are false negatives. 
- G00-03-0589290
- G00-21-0494028
- G00-21-2114990
- G00-32-0551737
- G00-86-3719816
- G00-92-2974327
- G00-99-0140748

 ### (3): SUGGESTION OF IMPROVEMENTS BY EXPLAINING FALSE POSITIVE AND FALSE NEGATIVE:

 ### False Positive example:
 - I have taken the G00-70-2681284 document to explain false positive. 
 - The reason it has been selected as a result of the IR system is, the document consists of all the query phrase tokens. 
 - As we have used RegexTokenizer() as an analyzer for text content in the schema for a query as well as documents, it transforms text into tokens. For our selected query it converts the phrase "Emergency and disaster preparedness assistance" into "Emergency", "and", "disaster", "preparedness" tokens and the same applies to the document text content.
 - As the document(G00-70-2681284) has all the words from query tokens. That's why it has been extracted as a required result.
#### REASON----------------------------------------------------------------
 - In the G00-92-2053892 document, the "preparedness" word only matches the URL of the article. "Preparedness" word can be found in the body text but it has its initial in uppercase which does not match with our query phrase. 

 ### False Negative example: 
  - I have taken the G00-92-2974327 document to explain false negative
  #### REASON-------------------------------------------------------------  
  - In this document, there are only two words match with the query phrase.

  - The document consists of "Disaster" and "Preparedness" and that too with the initial alphabet in uppercase. 
  - our query has only one word which has the initial alphabet in uppercase.("Emergency") whereas "disaster" and "preparedness" words have initial alphabets in lowercase. 
  - That's why the whoosh baseline system failed to identify words in the documents due to the uppercase alphabets in the document.
#### REASON----------------------------------------------------------------
  - Another reason for not selecting this document as a result of a system is that it has only two words from the query phrase. ("Disaster" and "Preparedness"). There is no mention of "Emergency" and "assistance" words. 
  - The query parser handles queries as conjunction. It treats the phrase as it is connected with "AND". So, the system returns only those documents which have all the words from the query phrase. The example of false negative does not contain all the words from the query.

  ### SUGGESTIONS FOR MODIFICATIONS:
  - Use of LowercaseFilter(). (To avoid the lowercase and uppercase problem.)
  - Use of qparser.OrGroup - to change the parser to use "OR". (so that it gives all the documents that may have any of the words from query phrase.) - Source: https://whoosh.readthedocs.io/en/latest/parsing.html
  - Use of field_boost for body content. ( to make terms found in the body field score higher compared to terms in the title field) - Source: https://whoosh.readthedocs.io/en/latest/schema.html
  
#### field_boost for body content decreases the performance to map 0.3461 and use of factory() with OrGroup also reduces the map to 0.3492. So, we are not going to include it in the performance improvement measures.

### Q3 (b): Write your code below

In [ ]:
# adding lowercase filter as an analyzer
LwrAnalyzer = RegexTokenizer() | LowercaseFilter()

In [ ]:
# Put your code for creating the index here (you can add more cells).
# first, define a Schema for the index
Schema_q3_1 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = LwrAnalyzer))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q3_1 = createIndex(Schema_q3_1)

# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3
addFilesToIndex(INDEX_Q3_1, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
og = qparser.OrGroup.factory(0.5) # factory value can be between 0 and 1
QP_Q3_1 = QueryParser("file_content", schema=INDEX_Q3_1.schema , group=og  ) # Replace None with your query parser for Q3
SEARCHER_Q3_1 = INDEX_Q3_1.searcher() # Replace None with your searcher for Q3
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q3_1, SEARCHER_Q3_1)

num_q                    1       1.0000
num_ret                  1       200.0000
num_rel                  1       5.0000
num_rel_ret              1       3.0000
map                      1       0.0313
gm_map                   1       -3.4629
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0625
iprec_at_recall_0.00     1       0.0667
iprec_at_recall_0.10     1       0.0667
iprec_at_recall_0.20     1       0.0667
iprec_at_recall_0.30     1       0.0667
iprec_at_recall_0.40     1       0.0667
iprec_at_recall_0.50     1       0.0275
iprec_at_recall_0.60     1       0.0275
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.06

In [ ]:
# Put your code for creating the index here (you can add more cells).
# first, define a Schema for the index
Schema_q3_2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = LwrAnalyzer, field_boost = 2.0))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q3_2 = createIndex(Schema_q3_2)

# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3
addFilesToIndex(INDEX_Q3_2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
#og = qparser.OrGroup.factory(0.5) # factory value can be between 0 and 1
QP_Q3_2 = QueryParser("file_content", schema=INDEX_Q3_2.schema , group=og  ) # Replace None with your query parser for Q3
SEARCHER_Q3_2 = INDEX_Q3_2.searcher() # Replace None with your searcher for Q3
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q3_2, SEARCHER_Q3_2)

num_q                    1       1.0000
num_ret                  1       200.0000
num_rel                  1       5.0000
num_rel_ret              1       3.0000
map                      1       0.0310
gm_map                   1       -3.4734
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0588
iprec_at_recall_0.00     1       0.0690
iprec_at_recall_0.10     1       0.0690
iprec_at_recall_0.20     1       0.0690
iprec_at_recall_0.30     1       0.0690
iprec_at_recall_0.40     1       0.0690
iprec_at_recall_0.50     1       0.0273
iprec_at_recall_0.60     1       0.0273
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.06

In [ ]:
# Put your code for creating the index here (you can add more cells).
# first, define a Schema for the index
Schema_q3 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = LwrAnalyzer))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q3 = createIndex(Schema_q3)

# Make sure you save the final index in the variable INDEX_Q3, your query parser in QP_Q3, and your searcher in SEARCHER_Q3

In [ ]:
addFilesToIndex(INDEX_Q3, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
# og = qparser.OrGroup.factory(0.5)

In [ ]:
#INDEX_Q3 = createIndex(Schema_q3) # Replace None with your index for Q3
QP_Q3 = QueryParser("file_content", schema=INDEX_Q3.schema , group=qparser.OrGroup  ) # Replace None with your query parser for Q3
SEARCHER_Q3 = INDEX_Q3.searcher() # Replace None with your searcher for Q3

In [ ]:
pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3)

num_q                    1       1.0000
num_ret                  1       200.0000
num_rel                  1       5.0000
num_rel_ret              1       3.0000
map                      1       0.0305
gm_map                   1       -3.4899
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0588
iprec_at_recall_0.00     1       0.0667
iprec_at_recall_0.10     1       0.0667
iprec_at_recall_0.20     1       0.0667
iprec_at_recall_0.30     1       0.0667
iprec_at_recall_0.40     1       0.0667
iprec_at_recall_0.50     1       0.0270
iprec_at_recall_0.60     1       0.0270
iprec_at_recall_0.70     1       0.0000
iprec_at_recall_0.80     1       0.0000
iprec_at_recall_0.90     1       0.0000
iprec_at_recall_1.00     1       0.0000
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.06

In [ ]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q3, SEARCHER_Q3, "16")

---------------------------Topic_id and Topic_phrase----------------------------------
16 Emergency and disaster preparedness assistance
---------------------------Return documents----------------------------------
16 Q0 G00-34-3591274 0 30.602030 test
16 Q0 G00-45-0006211 1 29.132549 test
16 Q0 G00-88-2853984 2 29.132549 test
16 Q0 G00-68-3661801 3 28.814357 test
16 Q0 G00-20-3839216 4 28.106687 test
16 Q0 G00-92-2053892 5 27.755464 test
16 Q0 G00-05-0719078 6 27.738594 test
16 Q0 G00-03-2245885 7 27.543609 test
16 Q0 G00-93-0870338 8 27.040545 test
16 Q0 G00-21-2114990 9 26.529445 test
16 Q0 G00-70-2681284 10 25.316882 test
16 Q0 G00-77-1693859 11 24.895912 test
16 Q0 G00-49-2630728 12 24.272907 test
16 Q0 G00-53-0263242 13 24.111588 test
16 Q0 G00-21-0494028 14 23.720758 test
16 Q0 G00-33-2857182 15 23.677108 test
16 Q0 G00-60-2564326 16 23.171653 test
16 Q0 G00-99-0140748 17 23.132433 test
16 Q0 G00-07-2244658 18 22.782481 test
16 Q0 G00-46-3010333 19 22.532484 test
16 Q0 G00-84-26

### Q3 (c): Provide answer to Q3 (c) here [markdown cell]

### MODIFICATIONS:
- Use of LowercaseFilter() with RegexTokenizer() 
- Use of qparser.OrGroup in Group

 #### These measures have increased the map for Q3 compared to Q2. The increased map for Q3 is 0.3542. 
 #### The system also returns all the "True Positives" however the number of "False Positives" has been increased significantly in the result after the modifications. 
 

 

### Q3 (d): Provide answer to Q3 (d) here [markdown cell]
### Yes

### Q3 (e): Provide answer to Q3 (e) here [markdown cell]
### Yes
- map for topic IDs 1,2,4,6,7,9,10,14,16,19, and 28 has been improved compared Q2. 
- map for topic IDs 22 and 26 has gotten worse.

### Q3 (f): Provide answer to Q3 (f) here [markdown cell]
### Yes, definitely the taken measures are good as it has increased the value of map but as we can observe that there are 3385 documents returned by the system for query 16 from which only 7 documents are "True Positives" or relevant documents. We need to reduce the number of false positives and increase the map also so that relevant documents get more score or highly ranked. 

## Question 4

## To get a map score, I have modified the pyTrecEval function a bit. This will provide ease for comparing different map values for different methods. 

In [ ]:
def pyTrecEval_map(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    # map_list = []
    
    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
           # print_line(measure, query_id, value)       #modification
    for measure in query_measures.keys():
        if measure == "map":
             
          print_line(
              measure,
              'all',
              pytrec_eval.compute_aggregated_measure(
                  measure,
                  [query_measures[measure]
                  for query_measures in results.values()]))

          # map_list.append(pytrec_eval.compute_aggregated_measure(measure,
          #                                                        [query_measures[measure]
          #                                                         for query_measures in results.values()]))

## To get a map score, I have modified the pyTrecEval function a bit. The below code returns only the score value. This will help in BM25F parameter tuning. 

In [ ]:
def pyTrecEval_map_value(topicFile, qrelsFile, queryParser, searcher):
    # Load topic file - a list of topics(search phrases) used for evalutation
    with open(topicFile, "r") as tf:
        topics = tf.read().splitlines()

    # create an output file to which we'll write our results
    tempOutputFile = tempfile.mkstemp()[1]
    with open(tempOutputFile, "w") as outputTRECFile:
        # for each evaluated topic:
        # build a query and record the results in the file in TREC_EVAL format
        for topic in topics:
            topic_id, topic_phrase = tuple(topic.split(" ", 1))
            #print(topic_id, topic_phrase)
            topicQuery = queryParser.parse(topic_phrase)
            topicResults = searcher.search(topicQuery, limit=None)
            for (docnum, result) in enumerate(topicResults):
                score = topicResults.score(docnum)
                #print("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
                outputTRECFile.write("%s Q0 %s %d %lf test\n" % (topic_id, os.path.basename(result["file_path"]), docnum, score))
    with open(qrelsFile, 'r') as f_qrel:
        qrel = pytrec_eval.parse_qrel(f_qrel)

    with open(tempOutputFile, 'r') as f_run:
        run = pytrec_eval.parse_run(f_run)

    evaluator = pytrec_eval.RelevanceEvaluator(
        qrel, pytrec_eval.supported_measures)

    results = evaluator.evaluate(run)
    def print_line(measure, scope, value):
        print('{:25s}{:8s}{:.4f}'.format(measure, scope, value))

    # map_list = []
    
    for query_id, query_measures in results.items():
        for measure, value in query_measures.items():
            if measure == "runid":
              continue
           # print_line(measure, query_id, value)       #modification
    for measure in query_measures.keys():
        if measure == "map":
          a= "{:.4f}".format(pytrec_eval.compute_aggregated_measure(
                  measure,
                  [query_measures[measure]
                  for query_measures in results.values()])) 
    
    return a
          # print_line(
          #     measure,
          #     'all',
          #     pytrec_eval.compute_aggregated_measure(
          #         measure,
          #         [query_measures[measure]
          #         for query_measures in results.values()]))
          

          # map_list.append(pytrec_eval.compute_aggregated_measure(measure,
          #                                                        [query_measures[measure]
          #                                                         for query_measures in results.values()]))

In [ ]:
# Put your code for creating the index here (you can add more cells).
# Make sure you save the final index in the variable INDEX_Q4, your query parser in QP_Q4, and your searcher in SEARCHER_Q4

Source : https://whoosh.readthedocs.io/en/latest/api/reading.html

In [ ]:
reader_q3 = INDEX_Q3.reader()
reader_q3.most_frequent_terms("file_content", number=5)

[(95991.0, b'the'),
 (62562.0, b'and'),
 (61428.0, b'of'),
 (45932.0, b'to'),
 (27700.0, b'in')]

# Method(1) : Use of StopFilter()

- As we can observe that most frequent words are prepositions and articles. We need to remove these words as they are not adding any value.  

In [ ]:
LwrStpaAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() 

Schema_q4_1 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = LwrStpaAnalyzer))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_1 = createIndex(Schema_q4_1)

addFilesToIndex(INDEX_Q4_1, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_1 = QueryParser("file_content", schema=INDEX_Q4_1.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_1 = INDEX_Q4_1.searcher() 

In [ ]:
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_1, SEARCHER_Q4_1)


map                      all     0.3716


In [ ]:
reader_q4_1 = INDEX_Q4_1.reader()
reader_q4_1.most_frequent_terms("file_content", number=5)

[(11400.0, b'information'),
 (5768.0, b'http'),
 (5657.0, b'site'),
 (5349.0, b'national'),
 (5170.0, b'page')]

- As we can see that afer application of stopfilter analizer, we removed articles and prepositon words. Now, in most frequent words it is showing us some useful words. 

# Method(2): Use of StopFilter() and IntrawordFilter()


In [ ]:
LwrStpIntraaAnalyzer = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter()

Schema_q4_2 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = LwrStpIntraaAnalyzer))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_2 = createIndex(Schema_q4_2)

addFilesToIndex(INDEX_Q4_2, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_2 = QueryParser("file_content", schema=INDEX_Q4_2.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_2 = INDEX_Q4_2.searcher()

In [ ]:
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_2, SEARCHER_Q4_2)

map                      all     0.3732


In [ ]:
nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [ ]:
# This filter will run for both the index and the query
from whoosh.analysis import Filter
class CustomFilter(Filter):
    is_morph = True
    def __init__(self, filterFunc, *args, **kwargs):
        self.customFilter = filterFunc
        self.args = args
        self.kwargs = kwargs
    def __eq__(self):
        return (other
                and self.__class__ is other.__class__)
    def __call__(self, tokens):
        for t in tokens:
            if t.mode == 'query': # if called by query parser
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t
            else: # == 'index' if called by indexer
                t.text = self.customFilter(t.text, *self.args, **self.kwargs)
                yield t

# Method(3): Use of NLTK's LancasterStemmer() as a custom filter 

In [ ]:

filter1 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | CustomFilter(LancasterStemmer().stem)

Schema_q4_3 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter1))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_3 = createIndex(Schema_q4_3)

addFilesToIndex(INDEX_Q4_3, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_3 = QueryParser("file_content", schema=INDEX_Q4_3.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_3 = INDEX_Q4_3.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_3, SEARCHER_Q4_3)

map                      all     0.3800


# Method(4): Use of NLTK's SnowballStemmer() as a custom filter 

In [ ]:
filter2 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | CustomFilter(SnowballStemmer("english").stem)

Schema_q4_4 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter2))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_4 = createIndex(Schema_q4_4)

addFilesToIndex(INDEX_Q4_4, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_4 = QueryParser("file_content", schema=INDEX_Q4_4.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_4 = INDEX_Q4_4.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_4, SEARCHER_Q4_4)

map                      all     0.3703


# Method(5): Use of NLTK's WordNetLemmatizer() as a custom filter 

In [ ]:
filter3 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | CustomFilter(WordNetLemmatizer().lemmatize)

Schema_q4_5 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter3))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_5 = createIndex(Schema_q4_5)

addFilesToIndex(INDEX_Q4_5, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_5 = QueryParser("file_content", schema=INDEX_Q4_5.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_5 = INDEX_Q4_5.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_5, SEARCHER_Q4_5)

map                      all     0.3743


# Method(6): Use of NLTK's WordNetLemmatizer().lemmatize(verb) as a custom filter 

In [ ]:
filter4 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | CustomFilter(WordNetLemmatizer().lemmatize, 'v')

Schema_q4_6 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter4))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_6 = createIndex(Schema_q4_6)

addFilesToIndex(INDEX_Q4_6, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_6 = QueryParser("file_content", schema=INDEX_Q4_6.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_6 = INDEX_Q4_6.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_6, SEARCHER_Q4_6)

map                      all     0.3754


# Method(7): Use of Stemfilter()

In [ ]:
filter5 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() 

Schema_q4_7 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter5))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_7 = createIndex(Schema_q4_7)

addFilesToIndex(INDEX_Q4_7, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_7 = QueryParser("file_content", schema=INDEX_Q4_7.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_7 = INDEX_Q4_7.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_7, SEARCHER_Q4_7)

map                      all     0.3716


# Method(8): Use of NLTK's LancasterStemmer() as a custom filter and StemFilter()

In [ ]:
filter6 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem)

Schema_q4_8 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter6))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_8 = createIndex(Schema_q4_8)

addFilesToIndex(INDEX_Q4_8, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_8 = QueryParser("file_content", schema=INDEX_Q4_8.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_8 = INDEX_Q4_8.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_8, SEARCHER_Q4_8)

map                      all     0.3894


# Method(9): Use of NgramFilter() with NLTK's LancasterStemmer() as a custom filter and StemFilter()

In [ ]:
filter7 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem)| NgramFilter(minsize=2, maxsize=4)

Schema_q4_9 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter7))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_9 = createIndex(Schema_q4_9)

addFilesToIndex(INDEX_Q4_9, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_9 = QueryParser("file_content", schema=INDEX_Q4_9.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_9 = INDEX_Q4_9.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_9, SEARCHER_Q4_9)

map                      all     0.3328


# Method(10): Use of NLTK's LancasterStemmer() and WordNetLemmatizer().lemmatize, 'v' as a custom filter and StemFilter()

In [ ]:
filter8 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem) | CustomFilter(WordNetLemmatizer().lemmatize, 'v')

Schema_q4_10 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter8))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_10 = createIndex(Schema_q4_10)

addFilesToIndex(INDEX_Q4_10, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_10 = QueryParser("file_content", schema=INDEX_Q4_10.schema , group=qparser.OrGroup  ) 
SEARCHER_Q4_10 = INDEX_Q4_10.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_10, SEARCHER_Q4_10)

map                      all     0.3894


# Method(11): Use of query.Variations for termclass in QueryParser

Source : https://whoosh.readthedocs.io/en/latest/stemming.html

In [ ]:
filter9 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem) 

Schema_q4_11 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter9))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_11 = createIndex(Schema_q4_11)

addFilesToIndex(INDEX_Q4_11, filesToIndex)

already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_11 = QueryParser("file_content", schema=INDEX_Q4_11.schema , group=qparser.OrGroup , termclass=query.Variations ) 
SEARCHER_Q4_11 = INDEX_Q4_11.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_11, SEARCHER_Q4_11)

map                      all     0.3833


# Method(12): Use of CharsetFilter()

Source: https://whoosh.readthedocs.io/en/latest/api/analysis.html

In [ ]:
filter10 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem) | CharsetFilter(accent_map)

Schema_q4_12 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter10))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_12 = createIndex(Schema_q4_12)

addFilesToIndex(INDEX_Q4_12, filesToIndex)


already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
QP_Q4_12 = QueryParser("file_content", schema=INDEX_Q4_12.schema , group=qparser.OrGroup ) 
#SQP_Q4_12.add_plugin(qparser.SequencePlugin())
SEARCHER_Q4_12 = INDEX_Q4_12.searcher()
pyTrecEval_map(TOPIC_FILE, QRELS_FILE, QP_Q4_12, SEARCHER_Q4_12)

map                      all     0.3894


# Method(13): Parameter tuning of BM25F scoring

In [ ]:
filter11 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem)


Schema_q4_13 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter11))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4_13 = createIndex(Schema_q4_13)

addFilesToIndex(INDEX_Q4_13, filesToIndex)


already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
b =  np.arange(0.1,5,0.1)
k1 = np.arange(0.1,5,0.1)



all_map = []
for i in b:
  for j in k1:
    w = scoring.BM25F(B=i, content_B=1.0, K1=j)
    QP_Q4_13 = QueryParser("file_content", schema=INDEX_Q4_13.schema , group=qparser.OrGroup ) 
    #SQP_Q4_12.add_plugin(qparser.SequencePlugin())
    SEARCHER_Q4_13 = INDEX_Q4_13.searcher(weighting=w)
    # print("B:" , i)
    # print("K1:", j)
    map = pyTrecEval_map_value(TOPIC_FILE, QRELS_FILE, QP_Q4_13, SEARCHER_Q4_13)

    all_map.append(map)

    


In [ ]:
len(all_map)

2401

In [ ]:
row = len(b)
column = len(k1)
matrix_map = pd.DataFrame(np.array(all_map).reshape(row,column), columns = k1 , index = b)

k1_opt = matrix_map.max(axis=0).idxmax()
b_opt =  matrix_map.max(axis=1).idxmax()
map_opt = matrix_map.max().max()

print("Optimal K1:", k1_opt)
print("Optimal B:", b_opt)
print("Max map:",map_opt)


Optimal K1: 2.5000000000000004
Optimal B: 0.5
Max map: 0.4101


In [ ]:
matrix_map.head(2)

,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5,1.6,1.7,1.8,1.9,2.0,2.1,2.2,2.3,2.4,2.5,2.6,2.7,2.8,2.9,3.0,3.1,3.2,3.3,3.4,3.5,3.6,3.7,3.8,3.9,4.0,4.1,4.2,4.3,4.4,4.5,4.6,4.7,4.8,4.9
0.1,0.3297,0.3293,0.3281,0.3277,0.3281,0.3287,0.3296,0.3298,0.3310,0.3319,0.3319,0.3319,0.3329,0.3330,0.3328,0.3323,0.3321,0.3337,0.3347,0.3345,0.3350,0.3362,0.3385,0.3385,0.3384,0.3392,0.3392,0.3392,0.3386,0.3499,0.3498,0.3498,0.3498,0.3492,0.3490,0.3491,0.3494,0.3493,0.3493,0.3491,0.3492,0.3488,0.3487,0.3489,0.3490,0.3489,0.3487,0.3487,0.3488
0.2,0.3325,0.3271,0.3256,0.3262,0.3265,0.3266,0.3331,0.3346,0.3358,0.3370,0.3372,0.3381,0.3403,0.3406,0.3410,0.3454,0.3786,0.3787,0.3787,0.3792,0.3790,0.3790,0.3790,0.3789,0.3791,0.3789,0.3781,0.3779,0.3776,0.3773,0.3772,0.3774,0.3773,0.3773,0.3773,0.3774,0.3777,0.3774,0.3773,0.3778,0.3779,0.3775,0.3772,0.3771,0.3770,0.3769,0.3770,0.3769,0.3768


In [ ]:
matrix_map.shape

(49, 49)

In [ ]:
filter11 = RegexTokenizer() | LowercaseFilter() | StopFilter() | IntraWordFilter() | StemFilter() | CustomFilter(LancasterStemmer().stem)


Schema_q4 = Schema(file_path = ID(stored=True),
                  file_content = TEXT(analyzer = filter11))

# now, create the index at the path INDEX_DIR based on the new schema

INDEX_Q4 = createIndex(Schema_q4)

addFilesToIndex(INDEX_Q4, filesToIndex)


already indexed: 1000
already indexed: 2000
already indexed: 3000
already indexed: 4000
done indexing.


In [ ]:
w = scoring.BM25F(B=0.5, content_B=1.0, K1=2.5)
QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema , group=qparser.OrGroup ) 

SEARCHER_Q4 = INDEX_Q4.searcher(weighting=w)

pyTrecEval(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4)


num_q                    1       1.0000
num_ret                  1       469.0000
num_rel                  1       5.0000
num_rel_ret              1       5.0000
map                      1       0.0648
gm_map                   1       -2.7359
Rprec                    1       0.0000
bpref                    1       0.0000
recip_rank               1       0.0526
iprec_at_recall_0.00     1       0.1034
iprec_at_recall_0.10     1       0.1034
iprec_at_recall_0.20     1       0.1034
iprec_at_recall_0.30     1       0.1034
iprec_at_recall_0.40     1       0.1034
iprec_at_recall_0.50     1       0.1034
iprec_at_recall_0.60     1       0.1034
iprec_at_recall_0.70     1       0.0494
iprec_at_recall_0.80     1       0.0494
iprec_at_recall_0.90     1       0.0446
iprec_at_recall_1.00     1       0.0446
P_5                      1       0.0000
P_10                     1       0.0000
P_15                     1       0.0000
P_20                     1       0.0500
P_30                     1       0.10

### Please answer the following questions here
(a) A clear list of all final modifications made. 
- Method(1) : Use of StopFilter()
- Method(2) : Use of StopFilter() and IntraWordFilter()
- Method(3) : Use of StopFilter(),IntraWordFilter() and LancasterStemmer()
- Method(4) : Use of StopFilter(),IntraWordFilter() and SnowballStemmer()
- Method(5) : Use of StopFilter(),IntraWordFilter() and WordNetLemmatizer()
- Method(6) : Use of StopFilter(),IntraWordFilter() and WordNetLemmatizer().lemmatize(verb)
- Method(7) : Use of StopFilter(),IntraWordFilter() and Stemfilter()
- Method(8) : Use of StopFilter(),IntraWordFilter(), Stemfilter() and LancasterStemmer()
- Method(9) : Use of StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and NgramFilter()
- Method(10) :  Use of StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and WordNetLemmatizer().lemmatize(verb)
- Method(11) :  Use of StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and query.Variations for termclass in QueryParser
- Method(12) :  Use of StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and CharsetFilter()
- Method(13) :  Use of StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and parameter tuning of BM25F scoring

(b)  Why each modification was made – how did it help? 
## Reasons:
### For Method(1): (map = 0.3716)
- As we can observe that most_freuent words in the corpus are prepositions and articles. These words are not adding any value to the system and instead reduced the quality of information retrieval. That is why I have applied it to remove stop words. After applying the filter map got increased from 0.3542.

### For Method(2): (map = 0.3732)
-  In this method, I implemented an intraword filter. It splits the intra-words based on delimiters and case transitions. It might be useful if the document has any intrawords. It helps to improve the system. map increased by 0.016 from the previous method. 

### Different stemming/lemmatization or combination of both:
### For Method(3 to 10): (map = 0.3833)
- I tried implementing different stemming, lemmatization, and combination of both filters. Stemming reduces the different forms of words by removing suffix into a single word. Lemmatization is also doing the same thing but it converts the word into its original form. By converting different forms of a particular word into its root word, it helps the IR system to find a query phrase more accurately. It helps in the selected query as one of the words from the query is "preparedness" stemming/lemmatize converts it into prepared or prepare which helps the system to search query fast and accurately. I am getting the highest map =0.3894 with the combination of StemFilter() from the whoosh analyzer and NLTK's stemming filter LancasterStemmer().

- I have used NgramFilter() from the whoosh analyzer. The purpose of using NgramFilter is just to try it and check the effect on the system. The NgramFilter tokenizes the query in grams with given min and max size. It creates different words from the given phrase so I was accepting a very low map but surprisingly it gave 0.3328 map.

### For Method(11): (map = 0.3833):
- query. Variations help to find variations of words in the documents. So for example the selected query (ID:16) has the word "preparedness". So there might be some variations after stemming like "prepar"or "prepared" or "prepare". So query. Variation acts like an "OR" query for the variations in the query phrase. It reduced the map to 0.3833. 

### For Method(12): (map = 0.3894)
- I used CharsetFilter(). This function helps to map a different accent in the text. For example "cafe`" and "cafe" are the same words but due to different words, the IR may fail to map the query words into documents. The application of this filter did not help to increase the map. The map remained the same.

### For Method(13): (map = 0.4101)
- In the IR system documents are scored based on relevance. In the whoosh searcher, it is being done by the BM25 scorer. Parameters of BM25 are B and K1 can be tuned to get max map. When documents are large and lengthy, in that case, it might be a case that a particular irrelevant word appears several times so K1 penalizes this term frequency whereas if a document is lengthy and it covers a particular subject then in there might be a case that a particularly relevant word appears several times. B helps not to affect these term frequencies and extract relevant documents. I am getting optimal B=0.5 and K1=2.5. This gives the highest map = 0.4101

### List of methods with its achieved map value:
- Method(1) : StopFilter() - 0.3716
- Method(2) : StopFilter() and IntraWordFilter() - 0.3732
- Method(3) : StopFilter(),IntraWordFilter() and LancasterStemmer() - 0.3800
- Method(4) : StopFilter(),IntraWordFilter() and SnowballStemmer() - .3703
- Method(5) : StopFilter(),IntraWordFilter() and WordNetLemmatizer() - 0.3743
- Method(6) : StopFilter(),IntraWordFilter() and WordNetLemmatizer().lemmatize(verb) - 0.3754
- Method(7) : StopFilter(),IntraWordFilter() and Stemfilter() - 0.3716
- Method(8) : StopFilter(),IntraWordFilter(), Stemfilter() and LancasterStemmer() - 0.3894
- Method(9) : StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and NgramFilter() - 0.3328
- Method(10) :  StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and WordNetLemmatizer().lemmatize(verb) - 0.3894
- Method(11) :  StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and query.Variations for termclass in QueryParser - 0.3833
- Method(12) :  StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and CharsetFilter() - 0.3894
- Method(13) :  StopFilter(),IntraWordFilter(), Stemfilter(),LancasterStemmer() and parameter tuning of BM25F scoring - 0.4101


# FINAL RESULT: All modifications help to reduce number of false positive documents from 3385 to 1111 and also increase map score.

(c)  The final  MAP  performance that these modifications attained.

# map = 0.4101

In [ ]:
printRelName(TOPIC_FILE, QRELS_FILE, QP_Q4, SEARCHER_Q4, "16")

---------------------------Topic_id and Topic_phrase----------------------------------
16 Emergency and disaster preparedness assistance
---------------------------Return documents----------------------------------
16 Q0 G00-34-3591274 0 33.285835 test
16 Q0 G00-68-3661801 1 32.091723 test
16 Q0 G00-70-2681284 2 30.989065 test
16 Q0 G00-21-2114990 3 29.932820 test
16 Q0 G00-03-2245885 4 29.562050 test
16 Q0 G00-77-1693859 5 29.505934 test
16 Q0 G00-84-2647789 6 29.409721 test
16 Q0 G00-05-0719078 7 28.983986 test
16 Q0 G00-21-0494028 8 28.681574 test
16 Q0 G00-86-3719816 9 27.847722 test
16 Q0 G00-53-0263242 10 27.804614 test
16 Q0 G00-60-2564326 11 27.789496 test
16 Q0 G00-33-2857182 12 27.664175 test
16 Q0 G00-92-2053892 13 27.375621 test
16 Q0 G00-45-0006211 14 27.309825 test
16 Q0 G00-88-2853984 15 27.309825 test
16 Q0 G00-93-0870338 16 25.793006 test
16 Q0 G00-75-3633903 17 24.606541 test
16 Q0 G00-09-1393028 18 24.453193 test
16 Q0 G00-56-2140972 19 24.220746 test
16 Q0 G00-46-30

In [ ]:
# w = scoring.BM25F(B=0.5, content_B=1.0, K1=2.5)
# #INDEX_Q4 = createIndex(Schema_q4_13) # Replace None with your index for Q4
# QP_Q4 = QueryParser("file_content", schema=INDEX_Q4.schema , group=qparser.OrGroup )  # Replace None with your query parser for Q4
# SEARCHER_Q4 = INDEX_Q4.searcher(weighting=w) # Replace None with your searcher for Q4

## Validation

In [ ]:
# Run the following cells to make sure your code returns the correct value types

In [ ]:
from whoosh.index import FileIndex
from whoosh.qparser import QueryParser
from whoosh.searching import Searcher
import os.path

### Q2 Validation

In [ ]:
assert(isinstance(INDEX_Q2, FileIndex)), "Index Type"
assert(isinstance(QP_Q2, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q2, Searcher)), "Searcher Type"
print("Q2 Types Validated")

Q2 Types Validated


### Q3 Validation

In [ ]:
assert(isinstance(INDEX_Q3, FileIndex)), "Index Type"
assert(isinstance(QP_Q3, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q3, Searcher)), "Searcher Type"
print("Q3 Types Validated")

Q3 Types Validated


### Q4 Validation

In [ ]:
assert(isinstance(INDEX_Q4, FileIndex)), "Index Type"
assert(isinstance(QP_Q4, QueryParser)), "Query Parser Type"
assert(isinstance(SEARCHER_Q4, Searcher)), "Searcher Type"
print("Q4 Types Validated")

Q4 Types Validated
